In [0]:
import tensorflow as tf 
tf.test.gpu_device_name()

''

# New Section

RUL


In [0]:
! pip install pandas

In [0]:
import io

In [0]:
import pandas as pd

In [0]:

!pwd

/content


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn import preprocessing

In [0]:
train_df = pd.read_csv('/content/drive/My Drive/sensor.csv',header=0)
train_df.drop(train_df.columns[[17]], axis=1, inplace=True)

In [0]:
'''
for i in range(2,54):
  #target = train_df[i]
  target = train_df.iloc[:, i]
  print(target)
  train_df = train_df.assign(InterpolateSLinear=train_df.target.interpolate(method='slinear'))
'''
#train_df = train_df.interpolate(method ='slinear', limit_direction ='forward') 
train_df = train_df.interpolate(method = 'linear' , order = 2 , limit_direction ='forward') 
#train_df.interpolate()
#train_df[1]

In [0]:
train_df.insert(54, "RUL", 0)  


In [0]:
train_df

,Unnamed: 0,timestamp,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12,sensor_13,sensor_14,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,sensor_22,sensor_23,sensor_24,sensor_25,sensor_26,sensor_27,sensor_28,sensor_29,sensor_30,sensor_31,sensor_32,sensor_33,sensor_34,sensor_35,sensor_36,sensor_37,sensor_38,sensor_39,sensor_40,sensor_41,sensor_42,sensor_43,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,sensor_50,sensor_51,machine_status,RUL
0,0,2018-04-01 00:00:00,2.465394,47.09201,53.211800,46.310760,634.375000,76.45975,13.41146,16.13136,15.56713,15.05353,37.22740,47.52422,31.11716,1.681353,419.5747,461.8781,466.3284,2.565284,665.3993,398.9862,880.0001,498.8926,975.9409,627.6740,741.7151,848.0708,429.0377,785.1935,684.9443,594.4445,682.8125,680.4416,433.7037,171.9375,341.9039,195.0655,90.32386,40.36458,31.51042,70.57291,30.989580,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,NORMAL,0
1,1,2018-04-01 00:01:00,2.465394,47.09201,53.211800,46.310760,634.375000,76.45975,13.41146,16.13136,15.56713,15.05353,37.22740,47.52422,31.11716,1.681353,419.5747,461.8781,466.3284,2.565284,665.3993,398.9862,880.0001,498.8926,975.9409,627.6740,741.7151,848.0708,429.0377,785.1935,684.9443,594.4445,682.8125,680.4416,433.7037,171.9375,341.9039,195.0655,90.32386,40.36458,31.51042,70.57291,30.989580,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,NORMAL,0
2,2,2018-04-01 00:02:00,2.444734,47.35243,53.211800,46.397570,638.888900,73.54598,13.32465,16.03733,15.61777,15.01013,37.86777,48.17723,32.08894,1.708474,420.8480,462.7798,459.6364,2.500062,666.2234,399.9418,880.4237,501.3617,982.7342,631.1326,740.8031,849.8997,454.2390,778.5734,715.6266,661.5740,721.8750,694.7721,441.2635,169.9820,343.1955,200.9694,93.90508,41.40625,31.25000,69.53125,30.468750,31.770830,41.66666,39.351852,65.39352,51.21528,38.194443,155.9606,67.12963,241.3194,203.7037,NORMAL,0
3,3,2018-04-01 00:03:00,2.460474,47.09201,53.168400,46.397568,628.125000,76.98898,13.31742,16.24711,15.69734,15.08247,38.57977,48.65607,31.67221,1.579427,420.7494,462.8980,460.8858,2.509521,666.0114,399.1046,878.8917,499.0430,977.7520,625.4076,739.2722,847.7579,474.8731,779.5091,690.4011,686.1111,754.6875,683.3831,446.2493,166.4987,343.9586,193.1689,101.04060,41.92708,31.51042,72.13541,30.468750,31.510420,40.88541,39.062500,64.81481,51.21528,38.194440,155.9606,66.84028,240.4514,203.1250,NORMAL,0
4,4,2018-04-01 00:04:00,2.445718,47.13541,53.211800,46.397568,636.458300,76.58897,13.35359,16.21094,15.69734,15.08247,39.48939,49.06298,31.95202,1.683831,419.8926,461.4906,468.2206,2.604785,663.2111,400.5426,882.5874,498.5383,979.5755,627.1830,737.6033,846.9182,408.8159,785.2307,704.6937,631.4814,766.1458,702.4431,433.9081,164.7498,339.9630,193.8770,101.70380,42.70833,31.51042,76.82291,30.989580,31.510420,41.40625,38.773150,65.10416,51.79398,38.773150,158.2755,66.55093,242.1875,201.3889,NORMAL,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220315,220315,2018-08-31 23:55:00,2.407350,47.69965,50.520830,43.142361,634.722229,64.59095,15.11863,16.65220,15.65393,15.16204,43.17085,54.16052,38.05424,13.265320,420.7993,463.2318,458.3615,2.499117,676.6655,405.7680,894.5920,543.5801,1109.5010,611.1745,700.5885,796.5964,692.1138,779.2067,485.0358,691.6666,974.9999,927.6135,477.3156,266.0334,578.5221,817.5707,0.00000,47.13541,29.16667,71.61458,30.468750,30.208330,38.28125,68.287030,52.37268,48.32176,41.087960,212.3843,153.64580,1000.0000,231.1921,NORMAL,0
220316,220316,2018-08-31 23:56:00,2.400463,47.69965,50.564240,43.142361,630.902771,65.83363,15.15480,16.70284,15.65393,15.11863,43.21038,54.52602,38.53485,13.242270,422.1567,463.1928,468.4388,2.618476,676.6547,

In [0]:
k = 0
for i in range(len(train_df)):
    if(train_df["machine_status"][i] == "BROKEN" ):
        count = 0
        for j in range(i,k-1,-1):
            train_df.at[j,'RUL']= count
            count = count + 1
        k = i

In [0]:
n = len(train_df)-1
while(train_df["machine_status"][n] != "BROKEN"):
    n -= 1   
train_df = train_df[:n+1]        
train_df["machine_status"][n+1] = 0    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_n

In [0]:
from pandas import read_csv, DataFrame
from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model

Using TensorFlow backend.


In [0]:
train_df.drop(['machine_status'], axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
train_df.drop(['Unnamed: 0'], axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
train_df

,timestamp,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12,sensor_13,sensor_14,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,sensor_22,sensor_23,sensor_24,sensor_25,sensor_26,sensor_27,sensor_28,sensor_29,sensor_30,sensor_31,sensor_32,sensor_33,sensor_34,sensor_35,sensor_36,sensor_37,sensor_38,sensor_39,sensor_40,sensor_41,sensor_42,sensor_43,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,sensor_50,sensor_51,RUL
0,2018-04-01 00:00:00,2.465394,47.092010,53.21180,46.310760,634.375000,76.45975,13.41146,16.13136,15.56713,15.05353,37.22740,47.52422,31.11716,1.681353,419.5747,461.8781,466.3284,2.565284,665.3993,398.9862,880.0001,498.8926,975.9409,627.6740,741.7151,848.0708,429.0377,785.1935,684.9443,594.4445,682.8125,680.4416,433.7037,171.9375,341.9039,195.0655,90.32386,40.36458,31.51042,70.57291,30.989580,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,17155
1,2018-04-01 00:01:00,2.465394,47.092010,53.21180,46.310760,634.375000,76.45975,13.41146,16.13136,15.56713,15.05353,37.22740,47.52422,31.11716,1.681353,419.5747,461.8781,466.3284,2.565284,665.3993,398.9862,880.0001,498.8926,975.9409,627.6740,741.7151,848.0708,429.0377,785.1935,684.9443,594.4445,682.8125,680.4416,433.7037,171.9375,341.9039,195.0655,90.32386,40.36458,31.51042,70.57291,30.989580,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,17154
2,2018-04-01 00:02:00,2.444734,47.352430,53.21180,46.397570,638.888900,73.54598,13.32465,16.03733,15.61777,15.01013,37.86777,48.17723,32.08894,1.708474,420.8480,462.7798,459.6364,2.500062,666.2234,399.9418,880.4237,501.3617,982.7342,631.1326,740.8031,849.8997,454.2390,778.5734,715.6266,661.5740,721.8750,694.7721,441.2635,169.9820,343.1955,200.9694,93.90508,41.40625,31.25000,69.53125,30.468750,31.770830,41.66666,39.351852,65.39352,51.21528,38.194443,155.9606,67.12963,241.3194,203.7037,17153
3,2018-04-01 00:03:00,2.460474,47.092010,53.16840,46.397568,628.125000,76.98898,13.31742,16.24711,15.69734,15.08247,38.57977,48.65607,31.67221,1.579427,420.7494,462.8980,460.8858,2.509521,666.0114,399.1046,878.8917,499.0430,977.7520,625.4076,739.2722,847.7579,474.8731,779.5091,690.4011,686.1111,754.6875,683.3831,446.2493,166.4987,343.9586,193.1689,101.04060,41.92708,31.51042,72.13541,30.468750,31.510420,40.88541,39.062500,64.81481,51.21528,38.194440,155.9606,66.84028,240.4514,203.1250,17152
4,2018-04-01 00:04:00,2.445718,47.135410,53.21180,46.397568,636.458300,76.58897,13.35359,16.21094,15.69734,15.08247,39.48939,49.06298,31.95202,1.683831,419.8926,461.4906,468.2206,2.604785,663.2111,400.5426,882.5874,498.5383,979.5755,627.1830,737.6033,846.9182,408.8159,785.2307,704.6937,631.4814,766.1458,702.4431,433.9081,164.7498,339.9630,193.8770,101.70380,42.70833,31.51042,76.82291,30.989580,31.510420,41.40625,38.773150,65.10416,51.79398,38.773150,158.2755,66.55093,242.1875,201.3889,17151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166436,2018-07-25 13:56:00,2.313889,45.833330,53.03819,43.923610,71.464119,73.00980,14.36632,16.21094,15.69734,15.12587,38.34975,49.88500,31.94489,12.285810,421.4294,458.9118,456.3463,2.459503,666.8698,398.0684,879.7014,536.2166,1091.1240,629.8835,740.9833,981.7558,497.2223,995.5433,563.5882,676.8519,949.9999,990.9057,564.7968,342.3690,498.5037,804.9069,55.82830,53.38541,41.40625,93.48958,36.197910,36.458332,49.47916,52.083330,51.79398,60.47454,45.138890,252.3148,69.15509,1000.0000,198.2060,4
166437,2018-07-25 13:57:00,2.315856,45.833332,53.03819,43.923611,3.336227,72.47576,14.33015,16.16030,15.76968,15.16204,38.58192,49.96759,31.46310,12.213550,420.6030,460.6981,467.5996,2.603880,665.2674,399.1180,879.2195,534.4346,1089.8550,628.6547,738.9200,979.7875,504.8127,1004.9950,

Multi class Classification using LSTM


In [0]:
period1 = 50
period2 = 300

In [0]:
train_df.insert(53, "lab_bin", 0)

In [0]:
for i in range(len(train_df)):
    if(train_df["RUL"][i] < period1):
        train_df.at[i,'lab_bin'] = 1    
    elif(train_df["RUL"][i] < period2):
        train_df.at[i,'lab_bin'] = 2


In [0]:
train_df    

,timestamp,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12,sensor_13,sensor_14,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,sensor_22,sensor_23,sensor_24,sensor_25,sensor_26,sensor_27,sensor_28,sensor_29,sensor_30,sensor_31,sensor_32,sensor_33,sensor_34,sensor_35,sensor_36,sensor_37,sensor_38,sensor_39,sensor_40,sensor_41,sensor_42,sensor_43,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,sensor_50,sensor_51,RUL,lab_bin
0,2018-04-01 00:00:00,2.465394,47.092010,53.21180,46.310760,634.375000,76.45975,13.41146,16.13136,15.56713,15.05353,37.22740,47.52422,31.11716,1.681353,419.5747,461.8781,466.3284,2.565284,665.3993,398.9862,880.0001,498.8926,975.9409,627.6740,741.7151,848.0708,429.0377,785.1935,684.9443,594.4445,682.8125,680.4416,433.7037,171.9375,341.9039,195.0655,90.32386,40.36458,31.51042,70.57291,30.989580,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,17155,0
1,2018-04-01 00:01:00,2.465394,47.092010,53.21180,46.310760,634.375000,76.45975,13.41146,16.13136,15.56713,15.05353,37.22740,47.52422,31.11716,1.681353,419.5747,461.8781,466.3284,2.565284,665.3993,398.9862,880.0001,498.8926,975.9409,627.6740,741.7151,848.0708,429.0377,785.1935,684.9443,594.4445,682.8125,680.4416,433.7037,171.9375,341.9039,195.0655,90.32386,40.36458,31.51042,70.57291,30.989580,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,17154,0
2,2018-04-01 00:02:00,2.444734,47.352430,53.21180,46.397570,638.888900,73.54598,13.32465,16.03733,15.61777,15.01013,37.86777,48.17723,32.08894,1.708474,420.8480,462.7798,459.6364,2.500062,666.2234,399.9418,880.4237,501.3617,982.7342,631.1326,740.8031,849.8997,454.2390,778.5734,715.6266,661.5740,721.8750,694.7721,441.2635,169.9820,343.1955,200.9694,93.90508,41.40625,31.25000,69.53125,30.468750,31.770830,41.66666,39.351852,65.39352,51.21528,38.194443,155.9606,67.12963,241.3194,203.7037,17153,0
3,2018-04-01 00:03:00,2.460474,47.092010,53.16840,46.397568,628.125000,76.98898,13.31742,16.24711,15.69734,15.08247,38.57977,48.65607,31.67221,1.579427,420.7494,462.8980,460.8858,2.509521,666.0114,399.1046,878.8917,499.0430,977.7520,625.4076,739.2722,847.7579,474.8731,779.5091,690.4011,686.1111,754.6875,683.3831,446.2493,166.4987,343.9586,193.1689,101.04060,41.92708,31.51042,72.13541,30.468750,31.510420,40.88541,39.062500,64.81481,51.21528,38.194440,155.9606,66.84028,240.4514,203.1250,17152,0
4,2018-04-01 00:04:00,2.445718,47.135410,53.21180,46.397568,636.458300,76.58897,13.35359,16.21094,15.69734,15.08247,39.48939,49.06298,31.95202,1.683831,419.8926,461.4906,468.2206,2.604785,663.2111,400.5426,882.5874,498.5383,979.5755,627.1830,737.6033,846.9182,408.8159,785.2307,704.6937,631.4814,766.1458,702.4431,433.9081,164.7498,339.9630,193.8770,101.70380,42.70833,31.51042,76.82291,30.989580,31.510420,41.40625,38.773150,65.10416,51.79398,38.773150,158.2755,66.55093,242.1875,201.3889,17151,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166436,2018-07-25 13:56:00,2.313889,45.833330,53.03819,43.923610,71.464119,73.00980,14.36632,16.21094,15.69734,15.12587,38.34975,49.88500,31.94489,12.285810,421.4294,458.9118,456.3463,2.459503,666.8698,398.0684,879.7014,536.2166,1091.1240,629.8835,740.9833,981.7558,497.2223,995.5433,563.5882,676.8519,949.9999,990.9057,564.7968,342.3690,498.5037,804.9069,55.82830,53.38541,41.40625,93.48958,36.197910,36.458332,49.47916,52.083330,51.79398,60.47454,45.138890,252.3148,69.15509,1000.0000,198.2060,4,1
166437,2018-07-25 13:57:00,2.315856,45.833332,53.03819,43.923611,3.336227,72.47576,14.33015,16.16030,15.76968,15.16204,38.58192,49.96759,31.46310,12.213550,420.6030,460.6981,467.5996,2.603880,665.2674,399.1180,879.2195,534.4346,1089.8550,628.6547,738.9200,979.

In [0]:
import numpy as np

import matplotlib.pyplot as plt

from sklearn import metrics

from sklearn import model_selection

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC

from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.model_selection import KFold

In [0]:
y = pd.get_dummies(train_df['lab_bin']).values
X = train_df.iloc[:, 1:52]

In [0]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

In [0]:
def calcp(a):
  l=[]
  for j in range(3): 
    summ = 0
    for i in range(3):
      summ = summ + a[i][j]
    l.append(a[j][j]/summ)
  return l

def calcr(a):
  l=[]
  for i in range(3):
    summ = 0
    for j in range(3):
      summ = summ + a[i][j]
    l.append(a[i][i]/summ)
  return l

In [0]:
# kf = KFold(n_splits=2)
# kf.get_n_splits(X)

# for train_index, test_index in kf.split(X):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     Y_train, Y_test = y[train_index], y[test_index]

#     sc_X = StandardScaler()
#     X_train =sc_X.fit_transform(X_train)
#     X_test = sc_X.transform(X_test)

    
#     classifier = MLPClassifier()
#     classifier.fit(X_train, Y_train)

#     y_pred = classifier.predict(X_test)

#     cm = confusion_matrix(Y_test, y_pred)
#     print(cm)
#     print(metrics.accuracy_score(Y_test, y_pred, normalize=True))
#     print(calcp(cm))
#     print(calcr(cm))

# # X.head()
# # X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.6, random_state = seed(20))


In [0]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [0]:

X.head()
X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size = 0.6, random_state = seed(20))

sc_X = StandardScaler()
X_train =sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)



In [0]:
n_features = len(X_train[0])
n_steps = 1

In [0]:
X_train= X_train.reshape((len(X_train), n_steps, n_features))
X_test=X_test.reshape((len(X_test), n_steps, n_features))

In [0]:
X_train.shape

(99864, 1, 51)

In [0]:

model = Sequential()
# model.add(LSTM(50, activation='relu', input_shape=(1, n_features)))
# model.add(Dense(1))
# model.compile(optimizer='adam', loss='mse')
model.add(LSTM(10, dropout=0.2, recurrent_dropout=0.2,input_shape=(1, n_features)))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=10, verbose=1)
ypred = model.predict(X_test, verbose=1)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/10





99864/99864 [==============================] - 9s 94us/step - loss: 0.0937 - acc: 0.9806
Epoch 2/10
99864/99864 [==============================] - 9s 89us/step - loss: 0.0373 - acc: 0.9905
Epoch 3/10
99864/99864 [==============================] - 9s 88us/step - loss: 0.0320 - acc: 0.9911
Epoch 4/10
99864/99864 [==============================] - 8s 84us/step - loss: 0.0290 - acc: 0.9918
Epoch 5/10
99864/99864 [==============================] - 9s 87us/step - loss: 0.0270 - acc: 0.9920
Epoch 6/10
99864/99864 [==============================] - 8s 84us/step - loss: 0.0251 - acc: 0.9925
Epoch 7/10
99864/99864 [==============================] - 9s 85us/step - loss: 0.0238 - acc: 0.9928
Epoch 8/10
99864/99864 [==============================] - 9s 88us/step - loss

In [0]:
y_pred= []
for i in ypred:
  y_pred.append(np.argmax(i))
print(y_pred)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [0]:
y_test= []
for i in Y_test:
  y_test.append(np.argmax(i))
print(y_test)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [0]:
# y_test= np.array(y_test)
# y_test.reshape(len(y_test),1)
# print(y_test)

In [0]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(metrics.accuracy_score(y_test, y_pred, normalize=True))
print(calcp(cm))
print(calcr(cm))

[[65725     2    34]
 [   43    10    80]
 [  137     0   546]]
0.9955540201571114
[0.9972687959942341, 0.8333333333333334, 0.8272727272727273]
[0.9994525630692964, 0.07518796992481203, 0.7994143484626647]


LSTM with multiple time steps as input for multiclass





In [0]:
train_df

,timestamp,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12,sensor_13,sensor_14,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,sensor_22,sensor_23,sensor_24,sensor_25,sensor_26,sensor_27,sensor_28,sensor_29,sensor_30,sensor_31,sensor_32,sensor_33,sensor_34,sensor_35,sensor_36,sensor_37,sensor_38,sensor_39,sensor_40,sensor_41,sensor_42,sensor_43,sensor_44,sensor_45,sensor_46,sensor_47,sensor_48,sensor_49,sensor_50,sensor_51,RUL,lab_bin
0,2018-04-01 00:00:00,2.465394,47.092010,53.21180,46.310760,634.375000,76.45975,13.41146,16.13136,15.56713,15.05353,37.22740,47.52422,31.11716,1.681353,419.5747,461.8781,466.3284,2.565284,665.3993,398.9862,880.0001,498.8926,975.9409,627.6740,741.7151,848.0708,429.0377,785.1935,684.9443,594.4445,682.8125,680.4416,433.7037,171.9375,341.9039,195.0655,90.32386,40.36458,31.51042,70.57291,30.989580,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,17155,0
1,2018-04-01 00:01:00,2.465394,47.092010,53.21180,46.310760,634.375000,76.45975,13.41146,16.13136,15.56713,15.05353,37.22740,47.52422,31.11716,1.681353,419.5747,461.8781,466.3284,2.565284,665.3993,398.9862,880.0001,498.8926,975.9409,627.6740,741.7151,848.0708,429.0377,785.1935,684.9443,594.4445,682.8125,680.4416,433.7037,171.9375,341.9039,195.0655,90.32386,40.36458,31.51042,70.57291,30.989580,31.770832,41.92708,39.641200,65.68287,50.92593,38.194440,157.9861,67.70834,243.0556,201.3889,17154,0
2,2018-04-01 00:02:00,2.444734,47.352430,53.21180,46.397570,638.888900,73.54598,13.32465,16.03733,15.61777,15.01013,37.86777,48.17723,32.08894,1.708474,420.8480,462.7798,459.6364,2.500062,666.2234,399.9418,880.4237,501.3617,982.7342,631.1326,740.8031,849.8997,454.2390,778.5734,715.6266,661.5740,721.8750,694.7721,441.2635,169.9820,343.1955,200.9694,93.90508,41.40625,31.25000,69.53125,30.468750,31.770830,41.66666,39.351852,65.39352,51.21528,38.194443,155.9606,67.12963,241.3194,203.7037,17153,0
3,2018-04-01 00:03:00,2.460474,47.092010,53.16840,46.397568,628.125000,76.98898,13.31742,16.24711,15.69734,15.08247,38.57977,48.65607,31.67221,1.579427,420.7494,462.8980,460.8858,2.509521,666.0114,399.1046,878.8917,499.0430,977.7520,625.4076,739.2722,847.7579,474.8731,779.5091,690.4011,686.1111,754.6875,683.3831,446.2493,166.4987,343.9586,193.1689,101.04060,41.92708,31.51042,72.13541,30.468750,31.510420,40.88541,39.062500,64.81481,51.21528,38.194440,155.9606,66.84028,240.4514,203.1250,17152,0
4,2018-04-01 00:04:00,2.445718,47.135410,53.21180,46.397568,636.458300,76.58897,13.35359,16.21094,15.69734,15.08247,39.48939,49.06298,31.95202,1.683831,419.8926,461.4906,468.2206,2.604785,663.2111,400.5426,882.5874,498.5383,979.5755,627.1830,737.6033,846.9182,408.8159,785.2307,704.6937,631.4814,766.1458,702.4431,433.9081,164.7498,339.9630,193.8770,101.70380,42.70833,31.51042,76.82291,30.989580,31.510420,41.40625,38.773150,65.10416,51.79398,38.773150,158.2755,66.55093,242.1875,201.3889,17151,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166436,2018-07-25 13:56:00,2.313889,45.833330,53.03819,43.923610,71.464119,73.00980,14.36632,16.21094,15.69734,15.12587,38.34975,49.88500,31.94489,12.285810,421.4294,458.9118,456.3463,2.459503,666.8698,398.0684,879.7014,536.2166,1091.1240,629.8835,740.9833,981.7558,497.2223,995.5433,563.5882,676.8519,949.9999,990.9057,564.7968,342.3690,498.5037,804.9069,55.82830,53.38541,41.40625,93.48958,36.197910,36.458332,49.47916,52.083330,51.79398,60.47454,45.138890,252.3148,69.15509,1000.0000,198.2060,4,1
166437,2018-07-25 13:57:00,2.315856,45.833332,53.03819,43.923611,3.336227,72.47576,14.33015,16.16030,15.76968,15.16204,38.58192,49.96759,31.46310,12.213550,420.6030,460.6981,467.5996,2.603880,665.2674,399.1180,879.2195,534.4346,1089.8550,628.6547,738.9200,979.

In [0]:
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


In [0]:
from numpy import array
from numpy import hstack

dataset = pd.concat([train_df.iloc[:, 1:52], train_df['lab_bin']], axis=1)
# define input sequence
dataset = dataset.to_numpy()
print(dataset)
# choose a number of time steps
n_steps = 100
# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(y)
y = pd.get_dummies(y).values
print(X.shape, y.shape)

n_features = X.shape[2]
# define model


[[   2.465394     47.09201      53.2118     ...  243.0556
   201.3889        0.        ]
 [   2.465394     47.09201      53.2118     ...  243.0556
   201.3889        0.        ]
 [   2.444734     47.35243      53.2118     ...  241.3194
   203.7037        0.        ]
 ...
 [   2.322743     45.83333      52.99479    ... 1000.
   206.8866        1.        ]
 [   2.315856     45.78993      53.03819    ... 1000.
   209.7801        1.        ]
 [   2.318808     45.83333206   52.99479    ... 1000.
   205.7292        1.        ]]
[0. 0. 0. ... 1. 1. 1.]
(166342, 100, 51) (166342, 3)


In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size = 0.6, random_state = seed(20))

In [0]:
from pandas import read_csv, DataFrame
from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras import metrics

In [0]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(3,activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=[metrics.mae,metrics.categorical_accuracy])
# fit model


In [0]:
model.fit(X_train, Y_train, epochs=1, verbose=1)


Epoch 1/1
99805/99805 [==============================] - 204s 2ms/step - loss: 2.2901 - mean_absolute_error: 0.0950 - categorical_accuracy: 0.8575


In [0]:
yhat = model.predict(X_test, verbose=0)
print(yhat)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [0]:
y_pred = []
for i in yhat:
  if i[0]:
    y_pred.append(0)
  elif i[1]:
    y_pred.append(1)
  else:
    y_pred.append(2)
y_test = []
for i in Y_test:
  if i[0]:
    y_test.append(0)
  elif i[1]:
    y_test.append(1)
  else:
    y_test.append(2)

In [0]:
from sklearn import metrics
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(metrics.accuracy_score(y_test, y_pred, normalize=True))
print(calcp(cm))
print(calcr(cm))

[[64263  1266   174]
 [  129     3     1]
 [  679    19     3]]
0.9659137021506831
[0.9875827941786663, 0.002329192546583851, 0.016853932584269662]
[0.9780831925482855, 0.022556390977443608, 0.0042796005706134095]
